# 📚 Context-Aware Book Summarization Tool

**Enhanced version with length control and improved prompting**

This notebook allows you to:
1. 📤 Upload a text file to summarize
2. 🤖 Select AI provider (Ollama / HuggingFace) and model
3. 📏 Choose summary length (Short / Medium / Long)
4. 📝 Generate high-quality summaries
5. 💾 Download the generated summary

---

## Step 1: Install Dependencies

In [ ]:
# Install required packages
!pip install -q transformers torch accelerate colorama ollama

print("✅ Dependencies installed successfully!")

## Step 2: Import Libraries and Setup

In [ ]:
import os
import re
import json
import time
import warnings
from pathlib import Path
from datetime import datetime

warnings.filterwarnings("ignore")

# Color support
try:
    from colorama import init, Fore, Style
    init(autoreset=True)
    C = True
except ImportError:
    C = False
    class Fore: RED = GREEN = YELLOW = CYAN = MAGENTA = RESET = ""
    class Style: BRIGHT = RESET_ALL = ""

# HuggingFace support
try:
    from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
    import torch
    HF_AVAILABLE = True
    print(f"{Fore.GREEN}✅ HuggingFace Transformers available{Style.RESET_ALL}")
except ImportError:
    HF_AVAILABLE = False
    print(f"{Fore.YELLOW}⚠️ HuggingFace Transformers not available{Style.RESET_ALL}")

# Ollama support check
try:
    import ollama
    OLLAMA_AVAILABLE = True
    print(f"{Fore.GREEN}✅ Ollama package available{Style.RESET_ALL}")
except ImportError:
    OLLAMA_AVAILABLE = False
    print(f"{Fore.YELLOW}⚠️ Ollama package not available{Style.RESET_ALL}")

# Check GPU availability
if torch.cuda.is_available():
    print(f"{Fore.GREEN}🚀 GPU Available: {torch.cuda.get_device_name(0)}{Style.RESET_ALL}")
else:
    print(f"{Fore.YELLOW}💻 Running on CPU{Style.RESET_ALL}")

print("\n✅ Setup complete!")

## Step 3: Define Summarization Engine

In [ ]:
# ---------------- LENGTH SPECIFICATIONS ----------------
LENGTH_SPECS = {
    "SHORT": {
        "chunk_target": "2-3 sentences",
        "final_ratio": "5-8%",
        "description": "Brief overview hitting only the most critical points"
    },
    "MEDIUM": {
        "chunk_target": "4-6 sentences", 
        "final_ratio": "10-15%",
        "description": "Balanced summary covering main ideas and key details"
    },
    "LONG": {
        "chunk_target": "8-12 sentences",
        "final_ratio": "20-25%", 
        "description": "Comprehensive summary preserving nuance and context"
    }
}

# ---------------- SUMMARY PROMPTS ----------------
SUMMARY_PROMPTS = {
    "BASIC": {
        "system": """You are a professional book summarizer focused on accuracy and clarity.

CORE PRINCIPLES:
- Preserve factual accuracy above all else
- Capture main ideas, events, and arguments in order
- Use clear, direct language
- NO interpretation, NO opinions, NO embellishment
- Maintain the author's voice and perspective""",
        
        "chunk_user": """TEXT TO SUMMARIZE:
\"\"\"
{chunk}
\"\"\"

CONTEXT FROM PREVIOUS SECTIONS:
{context}

TARGET LENGTH: {length_target}

Create a factual summary that:
1. Captures the key information in this section
2. Connects naturally with what came before
3. Preserves important names, events, and details
4. Uses approximately {length_target}
5. Maintains chronological order

Summary:""",

        "final_user": """You are creating a final cohesive summary from chunk summaries.

TARGET: {final_ratio} of original length
APPROACH: Create ONE flowing narrative (not a list of sections)

CHUNK SUMMARIES:
{summaries}

INSTRUCTIONS:
1. Combine all information into one seamless narrative
2. Remove ALL repetitions and redundancies
3. Maintain chronological/logical flow throughout
4. Preserve all key facts, names, events, and arguments
5. Connect ideas smoothly with transitions
6. Write as if summarizing the complete text directly
7. Target length: {final_ratio} of the original

Create the final summary:"""
    },

    "INTERMEDIATE": {
        "system": """You are an expert analytical summarizer who captures both content and context.

GOALS:
- Capture ideas, arguments, and their relationships
- Preserve narrative/logical flow and transitions
- Identify themes and patterns
- Connect new content with previous context
- Balance detail with conciseness
- Maintain the author's argumentative structure""",
        
        "chunk_user": """TEXT TO SUMMARIZE:
\"\"\"
{chunk}
\"\"\"

PREVIOUS CONTEXT:
{context}

TARGET LENGTH: {length_target}

Create a thematic summary that:
1. Identifies main ideas and how they connect
2. Preserves the logical flow and argumentation
3. Notes significant transitions or shifts
4. Maintains connection with previous context
5. Uses approximately {length_target}
6. Captures both explicit and implicit themes

Summary:""",

        "final_user": """Create a cohesive analytical summary from these chunk summaries.

TARGET: {final_ratio} of original length
FOCUS: Themes, arguments, and narrative flow

CHUNK SUMMARIES:
{summaries}

INSTRUCTIONS:
1. Synthesize into one flowing analytical narrative
2. Highlight thematic connections and patterns
3. Preserve the author's argumentative arc
4. Remove redundancies while keeping nuance
5. Show how ideas develop and connect
6. Write as a unified, coherent analysis
7. Target approximately {final_ratio} of original length

Create the cohesive summary:"""
    },

    "ADVANCED": {
        "system": """You are a senior literary analyst creating publication-quality summaries.

EXPERTISE:
- Capture intent, subtext, and rhetorical structure
- Explain WHY ideas matter, not just WHAT they are
- Identify authorial choices and their effects
- Preserve logical progression and development
- Synthesize rather than merely condense
- Think like a literary editor preparing reader guides""",
        
        "chunk_user": """TEXT TO ANALYZE:
\"\"\"
{chunk}
\"\"\"

NARRATIVE CONTEXT:
{context}

TARGET LENGTH: {length_target}

Create a sophisticated summary that:
1. Captures both surface content and deeper significance
2. Explains the function of this section in the larger work
3. Notes rhetorical choices and structural elements
4. Preserves the author's logic and progression
5. Identifies subtext and implications
6. Uses approximately {length_target}
7. Connects meaningfully with prior context

Analysis:""",

        "final_user": """Synthesize a sophisticated, publication-quality summary from these analytical summaries.

TARGET: {final_ratio} of original length
STANDARD: Literary analysis quality

CHUNK SUMMARIES:
{summaries}

INSTRUCTIONS:
1. Create ONE seamless narrative synthesis
2. Preserve the work's intellectual architecture
3. Show how ideas develop and interconnect
4. Capture both explicit content and deeper significance
5. Eliminate redundancy while preserving nuance
6. Write with the polish of a published analysis
7. Target approximately {final_ratio} of original length
8. Think like you're writing for a literary journal

Create the final synthesis:"""
    }
}

# ---------------- UTILITY FUNCTIONS ----------------
def detect_device():
    """Auto-detect available device (CUDA, ROCm, or CPU)."""
    try:
        if HF_AVAILABLE and torch.cuda.is_available():
            if hasattr(torch.version, 'hip') and torch.version.hip:
                print("🔍 ROCm (AMD GPU) detected")
                return "cuda"
            else:
                print("🔍 CUDA (NVIDIA GPU) detected")
                return "cuda"
    except:
        pass
    print("🔍 No GPU detected, using CPU")
    return "cpu"

def chunk_text(text, chunk_words=400, overlap=80):
    """Chunk text with overlap for context preservation"""
    words = text.split()
    chunks = []
    start = 0
    while start < len(words):
        end = start + chunk_words
        chunk = " ".join(words[start:end])
        chunks.append(chunk)
        start = end - overlap
    return chunks

def clean_output(text):
    """Remove thinking tags and code blocks from output"""
    text = re.sub(r'<think>.*?</think>', '', text, flags=re.DOTALL)
    text = re.sub(r'```.*?```', '', text, flags=re.DOTALL)
    text = re.sub(r'^(Summary:|Analysis:|Here is.*?:|Here\'s.*?:)\s*', '', text, flags=re.IGNORECASE)
    return text.strip()

def estimate_word_count(text):
    """Estimate word count from text"""
    return len(text.split())

print("✅ Summarization engine defined!")

## Step 4: Define Model Provider Class

In [ ]:
class Provider:
    """AI Model Provider for Ollama and HuggingFace"""
    
    def __init__(self, provider, model, device="auto"):
        self.provider = provider
        self.model = model
        self.device = detect_device() if device == "auto" else device
        self.pipeline = None

    def load(self):
        """Load the specified model"""
        if self.provider == "ollama":
            if not OLLAMA_AVAILABLE:
                raise RuntimeError("Ollama package not installed. Please install with: pip install ollama")
            try:
                ollama.show(self.model)
                print(f"{Fore.GREEN}✅ Ollama model '{self.model}' ready{Style.RESET_ALL}")
                return True
            except Exception as e:
                print(f"{Fore.YELLOW}⚠️ Model not found locally. Pulling {self.model}...{Style.RESET_ALL}")
                os.system(f"ollama pull {self.model}")
                return True

        # HuggingFace provider
        if not HF_AVAILABLE:
            raise RuntimeError("HuggingFace Transformers not installed")

        print(f"{Fore.CYAN}🔄 Loading HuggingFace model: {self.model}...{Style.RESET_ALL}")
        print("(This may take a few minutes for large models)")
        
        # Use appropriate dtype based on device
        if self.device == "cuda":
            torch_dtype = torch.float16
            device_map = "auto"
        else:
            torch_dtype = torch.float32
            device_map = None
        
        tok = AutoTokenizer.from_pretrained(self.model)
        mdl = AutoModelForCausalLM.from_pretrained(
            self.model,
            device_map=device_map,
            torch_dtype=torch_dtype
        )
        self.pipeline = pipeline("text-generation", model=mdl, tokenizer=tok)
        
        device_type = "GPU" if self.device == "cuda" else "CPU"
        print(f"{Fore.GREEN}✅ Model loaded on {device_type}{Style.RESET_ALL}")
        return True

    def generate(self, system, user, max_tokens=1500):
        """Generate text with the loaded model"""
        if self.provider == "ollama":
            out = ollama.chat(
                model=self.model,
                messages=[
                    {"role": "system", "content": system},
                    {"role": "user", "content": user}
                ],
                options={
                    "temperature": 0.2,
                    "num_ctx": 8192,
                    "num_predict": max_tokens
                }
            )
            return out["message"]["content"]

        # HuggingFace generation
        prompt = f"System:\n{system}\n\nUser:\n{user}\n\nAssistant:"
        r = self.pipeline(prompt, max_new_tokens=max_tokens, temperature=0.2)
        return r[0]["generated_text"].split("Assistant:")[-1]

print("✅ Provider class defined!")

## Step 5: Upload Your Text File 📤

In [ ]:
from google.colab import files

print("📤 Please upload your text file to summarize:")
print("(Supported formats: .txt)")
print()

uploaded = files.upload()

# Get the uploaded file
if uploaded:
    uploaded_filename = list(uploaded.keys())[0]
    input_text = uploaded[uploaded_filename].decode('utf-8')
    word_count = estimate_word_count(input_text)
    
    print()
    print(f"{Fore.GREEN}✅ File uploaded successfully!{Style.RESET_ALL}")
    print(f"📄 Filename: {uploaded_filename}")
    print(f"📊 Word count: {word_count:,} words")
    print(f"📝 Preview (first 500 chars):")
    print("-" * 50)
    print(input_text[:500] + "..." if len(input_text) > 500 else input_text)
else:
    print(f"{Fore.RED}❌ No file uploaded. Please run this cell again.{Style.RESET_ALL}")

## Step 6: Configure Summarization Options 🎛️

Run the cell below and use the interactive widgets to select your preferences.

In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# Create configuration widgets
provider_dropdown = widgets.Dropdown(
    options=['huggingface', 'ollama'],
    value='huggingface',
    description='AI Provider:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='400px')
)

model_text = widgets.Text(
    value='Qwen/Qwen2.5-1.5B-Instruct',
    placeholder='Enter model name',
    description='Model:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='400px')
)

length_dropdown = widgets.Dropdown(
    options=[
        ('🔹 SHORT - Brief overview (5-8% of original)', 'SHORT'),
        ('🔸 MEDIUM - Balanced summary (10-15% of original)', 'MEDIUM'),
        ('🔶 LONG - Comprehensive (20-25% of original)', 'LONG')
    ],
    value='MEDIUM',
    description='Length:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='500px')
)

tier_dropdown = widgets.Dropdown(
    options=[
        ('📘 BASIC - Factual, straightforward', 'BASIC'),
        ('📗 INTERMEDIATE - Thematic, analytical', 'INTERMEDIATE'),
        ('📕 ADVANCED - Publication-quality', 'ADVANCED')
    ],
    value='INTERMEDIATE',
    description='Quality Tier:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='500px')
)

chunk_words_slider = widgets.IntSlider(
    value=500,
    min=200,
    max=1000,
    step=50,
    description='Chunk Size:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='400px')
)

overlap_slider = widgets.IntSlider(
    value=100,
    min=20,
    max=200,
    step=10,
    description='Overlap:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='400px')
)

# Model suggestions based on provider
model_suggestions = widgets.HTML(
    value="""
    <div style='background: #f0f7ff; padding: 10px; border-radius: 8px; margin-top: 10px;'>
    <b>📋 Suggested HuggingFace Models:</b><br>
    • <code>Qwen/Qwen2.5-1.5B-Instruct</code> (Fast, good quality)<br>
    • <code>microsoft/Phi-3-mini-4k-instruct</code> (Balanced)<br>
    • <code>meta-llama/Llama-3.2-1B-Instruct</code> (Efficient)<br>
    • <code>google/gemma-2-2b-it</code> (High quality)<br>
    <br>
    <b>📋 Suggested Ollama Models:</b><br>
    • <code>llama3.1</code> (Recommended)<br>
    • <code>qwen2.5:7b</code> (Fast, accurate)<br>
    • <code>mistral</code> (Good balance)<br>
    • <code>phi3:mini</code> (Lightweight)
    </div>
    """
)

# Display configuration
print("🎛️ Configure Your Summarization Settings:")
print("=" * 50)
display(provider_dropdown)
display(model_text)
display(length_dropdown)
display(tier_dropdown)
display(chunk_words_slider)
display(overlap_slider)
display(model_suggestions)

print("\n✅ Configure your settings above, then run the next cell to generate the summary.")

## Step 7: Generate Summary 🚀

This cell will process your text and generate the summary based on your configuration.

In [ ]:
# Get configuration from widgets
selected_provider = provider_dropdown.value
selected_model = model_text.value
selected_length = length_dropdown.value
selected_tier = tier_dropdown.value
chunk_words = chunk_words_slider.value
overlap = overlap_slider.value

# Validate input
if 'input_text' not in dir() or not input_text:
    print(f"{Fore.RED}❌ No text file uploaded! Please run Step 5 first.{Style.RESET_ALL}")
else:
    print(f"{Fore.CYAN}{Style.BRIGHT}=== Book Summarization Tool ==={Style.RESET_ALL}")
    print(f"Provider: {selected_provider}")
    print(f"Model: {selected_model}")
    print(f"Tier: {selected_tier}")
    print(f"Length: {selected_length} ({LENGTH_SPECS[selected_length]['description']})")
    print(f"Chunk size: {chunk_words} words, Overlap: {overlap} words")
    print()

    # Initialize provider
    print(f"{Fore.YELLOW}🔄 Initializing AI provider...{Style.RESET_ALL}")
    prov = Provider(selected_provider, selected_model)
    prov.load()
    print()

    # Chunk text
    original_words = estimate_word_count(input_text)
    chunks = chunk_text(input_text, chunk_words, overlap)
    print(f"{Fore.GREEN}📊 Text loaded: {original_words:,} words → {len(chunks)} chunks{Style.RESET_ALL}")
    print()

    # Get prompts and length specs
    prompts = SUMMARY_PROMPTS[selected_tier]
    length_spec = LENGTH_SPECS[selected_length]

    # Process chunks
    context = ""
    chunk_summaries = []
    start_time = time.time()

    for i, chunk in enumerate(chunks, 1):
        print(f"{Fore.YELLOW}{Style.BRIGHT}► Processing chunk {i}/{len(chunks)}...{Style.RESET_ALL}", end=" ")
        
        chunk_prompt = prompts["chunk_user"].format(
            chunk=chunk,
            context=context[-2000:],
            length_target=length_spec["chunk_target"]
        )

        result = prov.generate(prompts["system"], chunk_prompt, max_tokens=800)
        result = clean_output(result)
        
        chunk_summaries.append(result)
        context += "\n\n" + result
        
        print(f"{Fore.GREEN}✓ ({estimate_word_count(result)} words){Style.RESET_ALL}")

    # Generate final cohesive summary
    print()
    print(f"{Fore.CYAN}{Style.BRIGHT}► Generating final cohesive summary...{Style.RESET_ALL}")
    
    final_prompt = prompts["final_user"].format(
        summaries="\n\n---\n\n".join(chunk_summaries),
        final_ratio=length_spec["final_ratio"]
    )
    
    final_summary = prov.generate(
        prompts["system"],
        final_prompt,
        max_tokens=2500
    )
    final_summary = clean_output(final_summary)
    
    # Calculate stats
    elapsed_time = time.time() - start_time
    final_words = estimate_word_count(final_summary)
    compression = (final_words / original_words) * 100
    
    print()
    print(f"{Fore.GREEN}{Style.BRIGHT}" + "=" * 50 + f"{Style.RESET_ALL}")
    print(f"{Fore.GREEN}{Style.BRIGHT}✅ SUMMARY COMPLETE!{Style.RESET_ALL}")
    print(f"{Fore.GREEN}{Style.BRIGHT}" + "=" * 50 + f"{Style.RESET_ALL}")
    print(f"📊 Original: {original_words:,} words")
    print(f"📝 Summary: {final_words:,} words ({compression:.1f}% of original)")
    print(f"⏱️ Time: {elapsed_time:.1f} seconds")
    print()
    print(f"{Fore.CYAN}=== GENERATED SUMMARY ==={Style.RESET_ALL}")
    print("-" * 50)
    print(final_summary)
    print("-" * 50)

## Step 8: Download Summary 💾

Run this cell to save and download your generated summary.

In [ ]:
from google.colab import files
from datetime import datetime

if 'final_summary' not in dir() or not final_summary:
    print(f"{Fore.RED}❌ No summary generated yet! Please run Step 7 first.{Style.RESET_ALL}")
else:
    # Generate output filename
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    base_name = uploaded_filename.rsplit('.', 1)[0] if 'uploaded_filename' in dir() else 'document'
    output_filename = f"{base_name}_summary_{selected_length.lower()}_{timestamp}.txt"
    
    # Create summary content with metadata
    summary_content = f"""================================================================================
SUMMARY GENERATED BY CONTEXT-AWARE BOOK SUMMARIZATION TOOL
================================================================================

Source File: {uploaded_filename if 'uploaded_filename' in dir() else 'Unknown'}
Generated: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
Provider: {selected_provider}
Model: {selected_model}
Quality Tier: {selected_tier}
Length Setting: {selected_length} ({LENGTH_SPECS[selected_length]['description']})

Original Word Count: {original_words:,}
Summary Word Count: {final_words:,}
Compression Ratio: {compression:.1f}%

================================================================================
SUMMARY
================================================================================

{final_summary}

================================================================================
END OF SUMMARY
================================================================================
"""
    
    # Save to file
    with open(output_filename, 'w', encoding='utf-8') as f:
        f.write(summary_content)
    
    print(f"{Fore.GREEN}✅ Summary saved!{Style.RESET_ALL}")
    print(f"📄 Filename: {output_filename}")
    print()
    print("📥 Starting download...")
    
    # Download the file
    files.download(output_filename)
    
    print(f"\n{Fore.GREEN}✅ Download initiated! Check your browser's downloads.{Style.RESET_ALL}")

---

## 🔄 Generate Another Summary (Optional)

Want to try different settings? Run the cell below to reset and start over.

In [ ]:
# Reset for new summarization
reset_confirm = input("⚠️ Are you sure you want to reset? This will clear the current summary. (yes/no): ")

if reset_confirm.lower() in ['yes', 'y']:
    # Clear variables
    if 'final_summary' in dir():
        del final_summary
    if 'chunk_summaries' in dir():
        del chunk_summaries
    if 'input_text' in dir():
        del input_text
    
    print(f"{Fore.GREEN}✅ Reset complete! Go back to Step 5 to upload a new file.{Style.RESET_ALL}")
else:
    print("Reset cancelled. Your current summary is preserved.")

---

## 📖 Quick Reference

### Length Options:
| Setting | Chunk Target | Final Ratio | Best For |
|---------|-------------|-------------|----------|
| **SHORT** | 2-3 sentences | 5-8% | Quick overviews, abstracts |
| **MEDIUM** | 4-6 sentences | 10-15% | Balanced summaries |
| **LONG** | 8-12 sentences | 20-25% | Detailed comprehension |

### Quality Tiers:
| Tier | Style | Best For |
|------|-------|----------|
| **BASIC** | Factual, straightforward | News, reports, documentation |
| **INTERMEDIATE** | Thematic, analytical | Books, essays, articles |
| **ADVANCED** | Publication-quality | Literary works, academic texts |

### Tips:
- 🔹 For faster processing, use smaller HuggingFace models or Ollama
- 🔹 Increase chunk overlap for better context preservation
- 🔹 Use ADVANCED tier for complex literary texts
- 🔹 GPU acceleration significantly speeds up HuggingFace models